## 实战案例：预测厄瓜多尔零售商的不同商店出售的数千种商品的单位销量

In [1]:
# 导入数据
import pandas as pd 
import numpy as np 
from sklearn.metrics import mean_squared_error 
from sklearn.preprocessing import LabelEncoder 
import lightgbm as lgb 
from datetime import date, timedelta


In [2]:
# 根目录
path = 'D:/Program Projects/Python Projects/temp/厄瓜多尔超市销量预测/input/'

In [3]:


# unit_sales 进行 log1p() 预处理，好处是可以 对偏度比较大的数据进行转化，将其压缩到一个较小的区间，
# 最后 log1p() 预处理 能起到平滑数据的作用。另外在评价指标部分也是对 unit_sales 进行同样的处理，这部分操作也是预处理。

# 另一个操作是对 date 进行处理，将表格文件中的时间字符串转换成日期格式。提 前处理不仅有便于后续操作，还能减少代码量。
df_train = pd.read_csv(path+'train.csv', converters={'unit_sales':lambda u: np.log1p(float(u)) if float(u) > 0 else 0}, parse_dates=["date"])

df_test = pd.read_csv(path + "test.csv",parse_dates=["date"]) 
items = pd.read_csv(path+'items.csv') 
stores = pd.read_csv(path+'stores.csv') 

# 类型转换 
df_train['onpromotion'] = df_train['onpromotion'].astype(bool) 
df_test['onpromotion'] = df_test['onpromotion'].astype(bool)

C:\Users\yx140\AppData\Local\Temp\ipykernel_77672\956254603.py:5: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv(path+'train.csv', converters={'unit_sales':lambda u: np.log1p(float(u)) if float(u) > 0 else 0}, parse_dates=["date"])


MemoryError: Unable to allocate 2.81 GiB for an array with shape (3, 125497040) and data type int64

In [ ]:
df_train

In [ ]:
df_test

In [ ]:
items

In [ ]:
stores

In [ ]:
# 数据集包含从 2013 年到 2017 年的数据，
# 时间跨度非常大，四年的发展过程中会产 生很多的不确定性。
# 在利用太久远的数据对未来进行预测时会产生一定的噪声，并 且会存在分布上的差异，
# 这一点在 11.2 节也可以发现。另外出于对性能的考虑，最 终仅使用 2017 年的数据作为训练集。执行下述代码过滤 2017 年之前的数据：
df_2017 = df_train.loc[df_train.date>='2017-01-01']
del df_train

In [ ]:
df_2017

In [ ]:
# 接下来进行基本的数据格式转换，并最终以店铺、商品和时间为索引，构造是否促 销的数据表，
# 以便进行与促销或者未促销相关的统计，这样的构造方式有利于之后 的特征提取。
promo_2017_train = df_2017.set_index(["store_nbr", "item_nbr", "date"])[["onpromotion"]].unstack(level=-1).fillna(False)
promo_2017_train.columns = promo_2017_train.columns.get_level_values(1)

promo_2017_test = df_test.set_index(["store_nbr", "item_nbr", "date"])[["onpromotion"]].unstack(level=-1).fillna(False)
promo_2017_test.columns = promo_2017_test.columns.get_level_values(1)
promo_2017_test = promo_2017_test.reindex(promo_2017_train.index).fillna(False)

promo_2017 = pd.concat([promo_2017_train, promo_2017_test], axis=1)
promo_2017 = promo_2017.astype('int')

In [ ]:
# 是否促销表
promo_2017

In [ ]:
df_2017 = df_2017.set_index(["store_nbr", "item_nbr", "date"])[["unit_sales"]].unstack(level=-1).fillna(0) 
df_2017.columns = df_2017.columns.get_level_values(1)

In [ ]:
# 单元销量表
df_2017

In [ ]:
# 历史平移特征和窗口统计特征是时间序列预测问题的核心特征，
# 这里仅简单地使用 历史平移特征（一个单位）和不同窗口大小的窗口统计特征作为基础特征。
# 下面实 现的是提取特征的通用代码：

def get_date_range(df, dt, forward_steps, periods, freq='D'): 
    return df[pd.date_range(start=dt-timedelta(days=forward_steps), periods=periods, freq=freq)]



In [ ]:
def prepare_dataset(t2017, is_train=True): 
    X = pd.DataFrame({ # 历史平移特征，前1、2、3 天的销量 
        "day_1_hist": get_date_range(df_2017, t2017, 1, 1).values.ravel(), 
        "day_2_hist": get_date_range(df_2017, t2017, 2, 1).values.ravel(), 
        "day_3_hist": get_date_range(df_2017, t2017, 3, 1).values.ravel(), })
    for i in [7, 14, 21, 30]: 
        # 窗口统计特征，销量diff/mean/meidan/max/min/std 
        X['diff_{}_day_mean'.format(i)] = get_date_range(df_2017, t2017, i, i).diff(axis=1).mean(axis=1).values 
        X['mean_{}_day'.format(i)] = get_date_range(df_2017, t2017, i, i).mean(axis=1).values 
        X['median_{}_day'.format(i)] = get_date_range(df_2017, t2017, i, i).mean(axis=1).values 
        X['max_{}_day'.format(i)] = get_date_range(df_2017, t2017, i, i).max(axis=1).values
        X['min_{}_day'.format(i)] = get_date_range(df_2017, t2017, i, i).min(axis=1).values 
        X['std_{}_day'.format(i)] = get_date_range(df_2017, t2017, i, i).min(axis=1).values
    for i in range(7): 
        # 前4、10 周每周的平均销量 
        X['mean_4_dow{}_2017'.format(i)] = get_date_range(df_2017, t2017, 28-i, 4, freq='7D').mean(axis=1).values 
        X['mean_10_dow{}_2017'.format(i)] = get_date_range(df_2017, t2017, 70-i, 10, freq='7D').mean(axis=1).values 
    for i in range(16): 
        # 未来16 天是否为促销日
        X["promo_{}".format(i)] = promo_2017[str(t2017 + timedelta(days=i))].values.astype(np.uint8) 
    if is_train: 
        y = df_2017[pd.date_range(t2017, periods=16)].values 
        return X, y 
    return X

In [ ]:
# 以7 月5 日后的第16 天作为最后一个训练集窗口，向前依次递推14 周得到14 个训练窗口的训练数据 
from tqdm import tqdm
X_l, y_l = [], [] 
t2017 = date(2017, 7, 5) 
n_range = 14 
for i in tqdm(range(n_range)): 
    delta = timedelta(days=7 * i) 
    X_tmp, y_tmp = prepare_dataset(t2017 - delta) 
    X_l.append(X_tmp) 
    y_l.append(y_tmp) 
X_train = pd.concat(X_l, axis=0) 
y_train = np.concatenate(y_l, axis=0) 
del X_l, y_l 

# 验证集取7 月26 日到8 月10 日的数据 
X_val, y_val = prepare_dataset(date(2017, 7, 26)) 

# 测试集取8 月16 日到8 月31 日的数据
X_test = prepare_dataset(date(2017, 8, 16), is_train=False)

In [ ]:
# 保存数据集
X_train.to_csv(path+'df_train.csv',index=False)
y_train.to_csv(path+'df_valid.csv')

In [ ]:
X_val
y_val

In [ ]:
from lightgbm import log_evaluation
params = {'num_leaves': 2 ** 5 - 1, 'objective': 'regression_l2', 'max_depth': 8, 'min_data_in_leaf': 50,
          'learning_rate': 0.05, 'feature_fraction': 0.75, 'bagging_fraction': 0.75, 'bagging_freq': 1, 'metric': 'l2',
          'num_threads': 4}
MAX_ROUNDS = 500
val_pred = []
test_pred = []
callbacks = [log_evaluation(period=100)]
for i in range(16): 
    print("====== Step %d ======" % (i + 1))
    dtrain = lgb.Dataset(X_train, label=y_train[:, i])
    dval = lgb.Dataset(X_val, label=y_val[:, i], reference=dtrain)
    bst = lgb.train(params, dtrain, num_boost_round=MAX_ROUNDS, valid_sets=[dtrain, dval], callbacks=callbacks)
    val_pred.append(bst.predict(X_val, num_iteration=bst.best_iteration or MAX_ROUNDS))
    test_pred.append(bst.predict(X_test, num_iteration=bst.best_iteration or MAX_ROUNDS))